In [31]:
import numpy as np
%matplotlib qt 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2

In [32]:
topdown_points = np.loadtxt('topdown1_undistorted_image_points_2D.txt')
fronton_points = np.loadtxt('fronton1_undistorted_image_points_2D.txt')
world_points = np.loadtxt('world_points_3D.txt')
topdown_camera = np.loadtxt('topdown1_new_camera_matrix.txt')
fronton_camera = np.loadtxt('fronton1_new_camera_matrix.txt')


In [33]:
# Convert to numpy arrays of type float32
topdown_points = np.array(topdown_points, dtype=np.float32)
fronton_points = np.array(fronton_points, dtype=np.float32)
world_points = np.array(world_points, dtype=np.float32)
topdown_camera = np.array(topdown_camera, dtype=np.float32)
fronton_camera = np.array(fronton_camera, dtype=np.float32)

# Use cv.solvePnP to find the rotation and translation vectors.

def get_rotation_and_translation_vectors(world_points, image_points, camera_matrix):
    """
    Returns the rotation and translation vectors for the given world and image points.
    :param world_points: The world points.
    :param image_points: The image points.
    :param camera_matrix: The camera matrix.
    :return: The rotation and translation vectors.
    """
    _, rvec, tvec = cv2.solvePnP(world_points, image_points, camera_matrix, None)
    # Since rvec is a rotation vector, we need to convert it to a rotation matrix.
    rmat, _ = cv2.Rodrigues(rvec)
    return rmat, tvec, rvec

topdown_rmat, topdown_tvec, topdown_rvec = get_rotation_and_translation_vectors(world_points, topdown_points, topdown_camera)
fronton_rmat, fronton_tvec, fronton_rvec = get_rotation_and_translation_vectors(world_points, fronton_points, fronton_camera)

In [34]:
def triangulation(camera1_coords, camera1_M, camera1_R, camera1_T, camera2_coords, camera2_M, camera2_R, camera2_T):
    """
    This function takes in a set of 2 points from 2 different cameras and finds the original point
    : param camera1_coords: A tuple of the (u,v) coordinates of the point in camera 1
    : param camera1_M: The camera matrix of camera 1
    : param camera1_R: The rotation vector of camera 1
    : param camera1_T: The translation vector of camera 1
    : param camera2_coords: A tuple of the (u,v) coordinates of the point in camera 2
    : param camera2_M: The camera matrix of camera 2
    : param camera2_R: The rotation vector of camera 2
    : param camera2_T: The translation vector of camera 2
    : return: The (x,y,z) coordinates of the original point
    """

    # Get the two key equations from camera1
    camera1_u, camera1_v = camera1_coords
    # Put the rotation and translation side by side and then multiply with camera matrix
    camera1_P = camera1_M.dot(np.column_stack((camera1_R,camera1_T)))
    # Get the two linearly independent equation referenced in the notes
    camera1_vect1 = camera1_v*camera1_P[2,:]-camera1_P[1,:]
    camera1_vect2 = camera1_P[0,:] - camera1_u*camera1_P[2,:]
    
    # Get the two key equations from camera2
    camera2_u, camera2_v = camera2_coords
    # Put the rotation and translation side by side and then multiply with camera matrix
    camera2_P = camera2_M.dot(np.column_stack((camera2_R,camera2_T)))
    # Get the two linearly independent equation referenced in the notes
    camera2_vect1 = camera2_v*camera2_P[2,:]-camera2_P[1,:]
    camera2_vect2 = camera2_P[0,:] - camera2_u*camera2_P[2,:]
    
    # Stack the 4 rows to create one 4x3 matrix
    full_matrix = np.row_stack((camera1_vect1, camera1_vect2, camera2_vect1, camera2_vect2))
    # The first three columns make up A and the last column is b
    A = full_matrix[:, :3]
    b = full_matrix[:, 3].reshape((4, 1))
    # Solve overdetermined system. The solution is the point in 3D space
    soln = np.linalg.inv(A.T.dot(A)).dot(A.T).dot(-b)
    return soln

triangulated_points = []
for i in range(len(topdown_points)):
    triangulated_points.append(triangulation(topdown_points[i], topdown_camera, topdown_rmat, topdown_tvec, fronton_points[i], fronton_camera, fronton_rmat, fronton_tvec))


In [35]:

# Get triangulation error

def get_triangulation_error(triangulated_points, world_points):
    """
    Returns the triangulation error.
    :param triangulated_points: The triangulated points.
    :param world_points: The world points.
    :return: The triangulation error.
    """
    return np.linalg.norm(triangulated_points[:,:,0] - world_points, axis=1).mean()

print('Triangulation error: {}'.format(get_triangulation_error(np.array(triangulated_points), world_points)))


Triangulation error: 0.03228534796266908


In [36]:
# Visualize the triangulated points

def visualize_triangulated_points(triangulated_points, world_points):
    """
    Visualizes the triangulated points.
    :param triangulated_points: The triangulated points.
    :param world_points: The world points.
    """
    plt.figure(figsize=(20,20),dpi=100)
    ax = plt.axes(projection='3d')
    X = np.array(triangulated_points)[:,0]
    Y = np.array(triangulated_points)[:,1]
    Z = np.array(triangulated_points)[:,2]
    max_range = np.array([X.max()-X.min(), Y.max()-Y.min(), Z.max()-Z.min()]).max() / 2.0

    mid_x = (X.max()+X.min()) * 0.5
    mid_y = (Y.max()+Y.min()) * 0.5
    mid_z = (Z.max()+Z.min()) * 0.5
    ax.set_xlim(mid_x - max_range, mid_x + max_range)
    ax.set_ylim(mid_y - max_range, mid_y + max_range)
    ax.set_zlim(mid_z - max_range, mid_z + max_range)
    ax.scatter3D(world_points[:,0], world_points[:,1], world_points[:,2], c='g', label='World Points')
    ax.scatter3D(X, Y, Z, c='r', label='Triangulated Points')
    plt.legend()
    plt.show()

visualize_triangulated_points(triangulated_points, world_points)


In [55]:
# Select new points to triangulate

imgpath = 'topdown1.png'
img = cv2.imread(imgpath)
topdown_image_points = []
fig = plt.figure(figsize=(80,120))

img = mpimg.imread(imgpath)

def onclick(event):
    ix, iy = event.xdata, event.ydata
    topdown_image_points.append([ix, iy])

cid = fig.canvas.mpl_connect('button_press_event', onclick)

imgplot = plt.imshow(img)
plt.show()


In [56]:

# This cell displays the points you have clicked.
N = len(topdown_image_points)
topdown_image_points = np.array(topdown_image_points)
fig = plt.figure(figsize=(10,15))

img = mpimg.imread(imgpath)
imgplot = plt.imshow(img)

colors = np.random.rand(N)
area = (30 * np.ones(N))**2 

plt.scatter(topdown_image_points[:,0], topdown_image_points[:,1], c=colors, s=area)
plt.show()


In [57]:
imgpath = 'fronton1.png'
img = cv2.imread(imgpath)
fronton_image_points = []
fig = plt.figure(figsize=(80,120))

img = mpimg.imread(imgpath)

def onclick(event):
    ix, iy = event.xdata, event.ydata
    fronton_image_points.append([ix, iy])

cid = fig.canvas.mpl_connect('button_press_event', onclick)

imgplot = plt.imshow(img)
plt.show()


In [58]:
N = len(fronton_image_points)
fronton_image_points = np.array(fronton_image_points)
fig = plt.figure(figsize=(10,15))

img = mpimg.imread(imgpath)
imgplot = plt.imshow(img)

colors = np.random.rand(N)
area = (30 * np.ones(N))**2 

plt.scatter(fronton_image_points[:,0], fronton_image_points[:,1], c=colors, s=area)
plt.show()

In [59]:
# Dewarp both points

topdown_old_camera = np.loadtxt('topdown1_camera_matrix.txt')
fronton_old_camera = np.loadtxt('fronton1_camera_matrix.txt')
topdown_distortion = np.loadtxt('topdown1_distortion_coefficients.txt')
fronton_distortion = np.loadtxt('fronton1_distortion_coefficients.txt')

dewarped_topdown_image_points = cv2.undistortPoints(np.array([topdown_image_points]), topdown_old_camera, topdown_distortion, P=topdown_camera).reshape(-1,2)
dewarped_fronton_image_points = cv2.undistortPoints(np.array([fronton_image_points]), fronton_old_camera, fronton_distortion, P=fronton_camera).reshape(-1,2)


new_triangulated_points = []
for i in range(len(topdown_image_points)):
    new_triangulated_points.append(triangulation(dewarped_topdown_image_points[i], topdown_camera, topdown_rmat, topdown_tvec, dewarped_fronton_image_points[i], fronton_camera, fronton_rmat, fronton_tvec))

# Visualize the new triangulated points
NET_Y = 0.76 + 3.96 + 1.98
NET_HEIGHT = 1.55
RIGHT_BOUNDARY_X = 0.46 + 2.59 + 2.59 + 0.46
new_world_points = np.array([[0 ,NET_Y, NET_HEIGHT],[ RIGHT_BOUNDARY_X,NET_Y ,NET_HEIGHT]])
print(len(new_triangulated_points))
visualize_triangulated_points(triangulated_points+new_triangulated_points, world_points)

# Get the new triangulation error

print('Triangulation error: {}'.format(get_triangulation_error(np.array(new_triangulated_points).T, world_points)))


2
Triangulation error: 6.444040643908214
